# DEPURACION DEFUNCIONES 1979-1991

In [11]:
# =====================================================
# ANÁLISIS EXPLORATORIO DE DATOS - VERSIÓN OPTIMIZADA
# =====================================================

## 1. IMPORTAR LIBRERIAS

In [4]:
### IMPORTAR LIBRERIAS ###
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

## 2. FUNCIONES

In [10]:
# =============================================================================
# 1. CONFIGURACIÓN Y FUNCIONES AUXILIARES
# =============================================================================

### 2.1. FUNCION PARA CARGAR DATOS DE DEFUNCION ###

In [6]:
def leer_archivos_defunciones(ruta_carpeta, año_inicio, año_fin):
    """Lee y unifica archivos de defunciones en un rango de años"""
    ruta = Path(ruta_carpeta)
    archivos = [arch for arch in ruta.glob("Defun*.txt") 
                if año_inicio <= int(arch.stem[-4:]) <= año_fin]
    
    dfs = []
    for archivo in tqdm(archivos, desc="Leyendo archivos"):
        try:
            df = pd.read_csv(archivo, sep='\t', encoding='utf-8', 
                           low_memory=False, dtype=str)
            df.columns = df.columns.str.upper()
            dfs.append(df)
        except Exception as e:
            print(f"Error en {archivo.name}: {e}")
    
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

### 2.2. FUNCION PARA CREAR MAPEO DE DESCRIPCIONES

In [7]:
def crear_mapeo_descripciones():
    """Crea diccionarios de mapeo para las descripciones"""
    return {
        'A_DEFUN': {
            '1': 'CABECERA MUNICIPAL',
            '2': 'RESTO',
            '3': 'SIN INFORMACIÓN'
        },
        'SEXO': {
            '1': 'MASCULINO',
            '2': 'FEMENINO'
        },
        'EST_CIVIL': {
            '1': 'SOLTERO',
            '2': 'CASADO',
            '3': 'VIUDO',
            '4': 'EN UNIÓN LIBRE, DIVORCIADO Y OTRO',
            '5': 'SIN INFORMACIÓN'
        },
        'SIT_DEFUN': {
            '1': 'HOSPITAL O CLINICA',
            '2': 'CASA',
            '3': 'OTRO SITIO',
            '4': 'SIN INFORMACIÓN'
        },
        'GRU_ED1': {
            '01': 'MENORES DE UN DÍA', '02': 'DE 1 A 6 DÍAS',
            '03': 'DE 7 A 29 DÍAS', '04': 'DE 1 A 5 MESES',
            '05': 'DE 6 A 11 MESES', '06': 'DE UN AÑO',
            '07': 'DE 2 A 4 AÑOS', '08': 'DE 5 A 9 AÑOS',
            '09': 'DE 10 A 14 AÑOS', '10': 'DE 15 A 19 AÑOS',
            '11': 'DE 20 A 24 AÑOS', '12': 'DE 25 A 29 AÑOS',
            '13': 'DE 30 A 34 AÑOS', '14': 'DE 35 A 39 AÑOS',
            '15': 'DE 40 A 44 AÑOS', '16': 'DE 45 A 49 AÑOS',
            '17': 'DE 50 A 54 AÑOS', '18': 'DE 55 A 59 AÑOS',
            '19': 'DE 60 A 64 AÑOS', '20': 'DE 65 A 69 AÑOS',
            '21': 'DE 70 A 74 AÑOS', '22': 'DE 75 A 79 AÑOS',
            '23': 'DE 80 A 84 AÑOS', '24': 'DE 85 Y MÁS AÑOS',
            '25': 'EDAD DESCONOCIDA'
        },
        'CONS_EXP': {
            '1': 'MEDICO TRATANTE',
            '2': 'MEDICO NO TRATANTE',
            '3': 'SIN CERTIFICACIÓN MÉDICA'
        }
    }

### 2.3. FUNCION PARA HOMOLOGAR DEPARTAMENTOS DE RESIDENCIA

In [8]:
def homologar_departamentos_residencia(df, divipola):
    """Homologa departamentos y municipios de residencia"""
    # Preparar códigos
    df['CODPTORE'] = df['CODPTORE'].astype(str).str.zfill(2)
    df['CODMUNRE'] = df['CODMUNRE'].astype(str).str.zfill(3)
    
    # Corregir código antiguo 83 -> 18 (Caquetá)
    df.loc[df['CODPTORE'] == '83', 'CODPTORE'] = '18'
    
    # Asegurar que no hay duplicados en divipola
    divipola_unico = divipola.drop_duplicates(subset=['COD_DPTO', 'COD_MUNIC'])
    
    # Merge principal
    df = df.merge(
        divipola_unico[['COD_DPTO', 'COD_MUNIC', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_MUNICIPIO']],
        left_on=['CODPTORE', 'CODMUNRE'],
        right_on=['COD_DPTO', 'COD_MUNIC'],
        how='left',
        suffixes=('', '_RESIDENCIA')
    )
    
    print(f"   Registros después del merge: {len(df):,}")
    
    # Completar nulos con cabeceras municipales
    cabeceras = divipola_unico[divipola_unico['COD_MUNIC'] == '001'][
        ['COD_DPTO', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_MUNICIPIO']
    ].drop_duplicates(subset=['COD_DPTO']).rename(columns={
        'NOMBRE_DEPARTAMENTO': 'DEPT_CAB',
        'NOMBRE_MUNICIPIO': 'MUNIC_CAB'
    })
    
    df = df.merge(cabeceras, left_on='CODPTORE', right_on='COD_DPTO', 
                  how='left', suffixes=('', '_y'))
    
    print(f"   Registros después de cabeceras: {len(df):,}")
    
    mask = df['NOMBRE_DEPARTAMENTO_RESIDENCIA'].isna()
    df.loc[mask, 'NOMBRE_DEPARTAMENTO_RESIDENCIA'] = df.loc[mask, 'DEPT_CAB']
    df.loc[mask, 'NOMBRE_MUNICIPIO_RESIDENCIA'] = df.loc[mask, 'MUNIC_CAB']
    
    # Marcar los que no se encontraron
    df['NOMBRE_DEPARTAMENTO_RESIDENCIA'].fillna('NO REGISTRA', inplace=True)
    df['NOMBRE_MUNICIPIO_RESIDENCIA'].fillna('NO REGISTRA', inplace=True)
    
    # Limpiar columnas temporales
    cols_drop = ['DEPT_CAB', 'MUNIC_CAB', 'COD_DPTO_y', 'COD_DPTO_RESIDENCIA', 
                 'COD_MUNIC_RESIDENCIA']
    df.drop([c for c in cols_drop if c in df.columns], axis=1, inplace=True)
    
    return df

### 2.4. FUNCION PARA BUSCAR CODIGOS CIE

In [9]:
def buscar_cie9_optimizado(c_bas1_val, cie_dict):
    """Búsqueda optimizada de códigos CIE-9 usando diccionario"""
    if pd.isna(c_bas1_val):
        return None
    
    c_bas1_str = str(c_bas1_val).strip()
    
    # Búsqueda directa
    if c_bas1_str in cie_dict:
        return cie_dict[c_bas1_str]
    
    # Solo para códigos numéricos
    if not c_bas1_str.isdigit():
        return None
    
    # Variaciones con prefijos (E, V, M, N)
    for prefijo in ['E', 'V', 'M', 'N']:
        # Con 4 dígitos
        if len(c_bas1_str) == 4:
            codigo = f"{prefijo}{c_bas1_str}"
            if codigo in cie_dict:
                return cie_dict[codigo]
        
        # Con 3 primeros dígitos
        if len(c_bas1_str) >= 3:
            codigo = f"{prefijo}{c_bas1_str[:3]}"
            if codigo in cie_dict:
                return cie_dict[codigo]
        
        # Sin ceros a la izquierda
        sin_ceros = c_bas1_str.lstrip('0')
        if sin_ceros and sin_ceros != c_bas1_str:
            if len(sin_ceros) >= 3:
                codigo = f"{prefijo}{sin_ceros[:3]}"
                if codigo in cie_dict:
                    return cie_dict[codigo]
    
    return None

### 2.5. FUNCION PARA CARGAR ARCHIVO LISTA 105 (CAUSAS DEFUNCION)

In [15]:
def leer_lista_105(ruta_archivo):
    """
    Lee el archivo de la Lista 105 de Colombia para tabulación de mortalidad.
    
    Parámetros:
    -----------
    ruta_archivo : str
        Ruta al archivo Excel (ej: "data/raw/Referenciales/Lista_105_Colombia_CIE9-y-CIE10.xls")
    
    Retorna:
    --------
    pd.DataFrame con columnas:
        - No_Lista: Número de la lista
        - Causa: Descripción de la causa
        - Codigos_CIE10: Códigos CIE-10
        - Codigos_CIE9: Códigos CIE-9
    """
    
    print(f"Leyendo archivo: {ruta_archivo}")
    
    # Leer el archivo Excel
    df = pd.read_excel(ruta_archivo, dtype=str)
    
    # Mostrar columnas detectadas
    print(f"Columnas detectadas: {list(df.columns)}")
    
    # Buscar la fila donde comienzan los datos (después del encabezado "LISTA COLOMBIA 105...")
    inicio_datos = None
    for idx, row in df.iterrows():
        if 'No. Lista' in str(row.values) or 'No.Lista' in str(row.values):
            inicio_datos = idx
            break
    
    if inicio_datos is None:
        # Intentar lectura alternativa
        print("Buscando encabezados de forma alternativa...")
        df = pd.read_excel(ruta_archivo, header=None, dtype=str)
        
        for idx, row in df.iterrows():
            row_str = ' '.join([str(x) for x in row.values if pd.notna(x)])
            if 'No. Lista' in row_str or 'No.Lista' in row_str:
                inicio_datos = idx
                break
    
    if inicio_datos is not None:
        print(f"Datos encontrados a partir de la fila: {inicio_datos}")
        # Leer desde la fila de inicio
        df = pd.read_excel(ruta_archivo, header=inicio_datos, dtype=str)
    
    # Limpiar nombres de columnas
    df.columns = df.columns.astype(str).str.strip()
    
    # Identificar las columnas correctas
    col_numero = None
    col_causa = None
    col_cie10 = None
    col_cie9 = None
    
    for col in df.columns:
        col_lower = col.lower()
        if 'no' in col_lower and 'lista' in col_lower:
            col_numero = col
        elif 'causa' in col_lower:
            col_causa = col
        elif 'cie-10' in col_lower or 'cie10' in col_lower:
            col_cie10 = col
        elif 'cie-9' in col_lower or 'cie9' in col_lower:
            col_cie9 = col
    
    print(f"Columnas identificadas:")
    print(f"  - Número: {col_numero}")
    print(f"  - Causa: {col_causa}")
    print(f"  - CIE-10: {col_cie10}")
    print(f"  - CIE-9: {col_cie9}")
    
    # Crear DataFrame normalizado
    df_limpio = pd.DataFrame()
    
    if all([col_numero, col_causa, col_cie10, col_cie9]):
        df_limpio['No_Lista'] = df[col_numero]
        df_limpio['Causa'] = df[col_causa]
        df_limpio['Codigos_CIE10'] = df[col_cie10]
        df_limpio['Codigos_CIE9'] = df[col_cie9]
    else:
        # Intento por posición de columnas
        print("Usando posición de columnas (A, B, C, D)")
        df_limpio['No_Lista'] = df.iloc[:, 0]
        df_limpio['Causa'] = df.iloc[:, 1]
        df_limpio['Codigos_CIE10'] = df.iloc[:, 2]
        df_limpio['Codigos_CIE9'] = df.iloc[:, 3]
    
    # Eliminar filas vacías
    df_limpio = df_limpio.dropna(subset=['No_Lista'], how='all')
    df_limpio = df_limpio[df_limpio['No_Lista'].notna()]
    
    # Limpiar espacios
    for col in df_limpio.columns:
        df_limpio[col] = df_limpio[col].astype(str).str.strip()
    
    # Eliminar filas que no tienen número de lista válido
    df_limpio = df_limpio[df_limpio['No_Lista'].str.isdigit()]
    
    # Convertir número de lista a entero
    df_limpio['No_Lista'] = df_limpio['No_Lista'].astype(int)
    
    print(f"\n✅ Archivo procesado exitosamente")
    print(f"   Total de causas: {len(df_limpio)}")
    print(f"   Rango de listas: {df_limpio['No_Lista'].min()} - {df_limpio['No_Lista'].max()}")
    
    return df_limpio

### 2.6. FUNCION PARA EXPANDIR CODIGOS CIE

In [16]:
def expandir_codigos_cie(df_lista105, columna_codigo='Codigos_CIE9'):
    """
    Expande los rangos de códigos CIE a códigos individuales.
    
    Ejemplo: "001-009" se expande a ["001", "002", ..., "009"]
    
    Parámetros:
    -----------
    df_lista105 : pd.DataFrame
        DataFrame con la Lista 105
    columna_codigo : str
        Nombre de la columna con los códigos ('Codigos_CIE9' o 'Codigos_CIE10')
    
    Retorna:
    --------
    pd.DataFrame con una fila por cada código individual
    """
    
    registros = []
    
    for _, row in df_lista105.iterrows():
        codigos_str = str(row[columna_codigo])
        
        if pd.isna(codigos_str) or codigos_str == 'nan':
            continue
        
        # Separar por comas
        codigos = [c.strip() for c in codigos_str.split(',')]
        
        for codigo in codigos:
            # Si es un rango (ej: "001-009")
            if '-' in codigo:
                partes = codigo.split('-')
                if len(partes) == 2:
                    inicio, fin = partes
                    inicio = inicio.strip()
                    fin = fin.strip()
                    
                    # Intentar expandir el rango
                    try:
                        # Extraer la parte numérica
                        prefijo_inicio = ''.join([c for c in inicio if not c.isdigit()])
                        num_inicio = int(''.join([c for c in inicio if c.isdigit()]))
                        
                        prefijo_fin = ''.join([c for c in fin if not c.isdigit()])
                        num_fin = int(''.join([c for c in fin if c.isdigit()]))
                        
                        # Generar códigos en el rango
                        for num in range(num_inicio, num_fin + 1):
                            codigo_expandido = f"{prefijo_inicio}{str(num).zfill(len(str(num_inicio)))}"
                            registros.append({
                                'No_Lista': row['No_Lista'],
                                'Causa': row['Causa'],
                                'Codigo': codigo_expandido
                            })
                    except:
                        # Si falla, agregar el código completo sin expandir
                        registros.append({
                            'No_Lista': row['No_Lista'],
                            'Causa': row['Causa'],
                            'Codigo': codigo
                        })
            else:
                # Código individual
                registros.append({
                    'No_Lista': row['No_Lista'],
                    'Causa': row['Causa'],
                    'Codigo': codigo
                })
    
    df_expandido = pd.DataFrame(registros)
    
    print(f"\n✅ Códigos expandidos: {len(df_expandido)} registros")
    
    return df_expandido

### 2.7. FUNCION PARA HOMOLOGAR CAUSA DEFUNCION DE LA LISTA 105

In [19]:
def homologar_causa_lista105(df_defunciones, df_lista105, col_codigo='CAU_HOMOL', 
                              crear_columna='CAU_HOMOL_DESC'):
    """
    Homologa el campo CAU_HOMOL con la Lista 105 de Colombia.
    
    Parámetros:
    -----------
    df_defunciones : pd.DataFrame
        DataFrame con las defunciones que tiene el campo CAU_HOMOL
    df_lista105 : pd.DataFrame
        DataFrame con la Lista 105 (resultado de leer_lista_105)
    col_codigo : str
        Nombre de la columna con el código de la lista 105 en df_defunciones (default: 'CAU_HOMOL')
    crear_columna : str
        Nombre de la columna a crear con la descripción (default: 'CAU_HOMOL_DESC')
    
    Retorna:
    --------
    pd.DataFrame con columna adicional:
        - {crear_columna}: Descripción de la causa según lista 105 (campo 'Causa')
    """
    
    print(f"\nHomologando CAU_HOMOL con Lista 105 de Colombia...")
    print(f"   Campo a cruzar: {col_codigo}")
    print(f"   Campo a crear: {crear_columna}")
    
    # Preparar datos para el merge
    # Asegurar que ambos campos sean string y estén limpios
    df_defunciones[col_codigo] = df_defunciones[col_codigo].astype(str).str.strip()
    df_lista105['No_Lista'] = df_lista105['No_Lista'].astype(str).str.strip()
    
    # Crear diccionario de mapeo para mayor eficiencia
    dict_lista105 = dict(zip(df_lista105['No_Lista'], df_lista105['Causa']))
    
    print(f"   Total códigos en Lista 105: {len(dict_lista105)}")
    print(f"   Ejemplo de mapeo: {list(dict_lista105.items())[:3]}")
    
    # Primera búsqueda: mapeo directo
    print(f"\n   Paso 1: Búsqueda directa...")
    df_defunciones[crear_columna] = df_defunciones[col_codigo].map(dict_lista105)
    
    coincidencias_directas = df_defunciones[crear_columna].notna().sum()
    print(f"   Coincidencias directas: {coincidencias_directas:,}")
    
    # Segunda búsqueda: quitar ceros a la izquierda para los no encontrados
    sin_coinc = df_defunciones[crear_columna].isna()
    if sin_coinc.sum() > 0:
        print(f"\n   Paso 2: Búsqueda sin ceros iniciales para {sin_coinc.sum():,} registros...")
        
        # Crear versión sin ceros a la izquierda
        df_defunciones.loc[sin_coinc, 'CAU_HOMOL_TEMP'] = (
            df_defunciones.loc[sin_coinc, col_codigo]
            .str.lstrip('0')  # Quitar ceros a la izquierda
        )
        
        # Aplicar mapeo con códigos sin ceros
        df_defunciones.loc[sin_coinc, crear_columna] = (
            df_defunciones.loc[sin_coinc, 'CAU_HOMOL_TEMP'].map(dict_lista105)
        )
        
        # Eliminar columna temporal
        df_defunciones.drop('CAU_HOMOL_TEMP', axis=1, inplace=True, errors='ignore')
        
        nuevas_coinc = df_defunciones.loc[sin_coinc, crear_columna].notna().sum()
        print(f"   Nuevas coincidencias: {nuevas_coinc:,}")
    
    # Estadísticas finales
    coincidencias_totales = df_defunciones[crear_columna].notna().sum()
    porcentaje = (coincidencias_totales / len(df_defunciones)) * 100
    
    print(f"\n✅ Homologación completada:")
    print(f"   Total registros: {len(df_defunciones):,}")
    print(f"   Coincidencias totales: {coincidencias_totales:,} ({porcentaje:.2f}%)")
    print(f"   Sin coincidencia: {(len(df_defunciones) - coincidencias_totales):,}")
    
    # Mostrar algunos valores únicos de CAU_HOMOL sin coincidencia
    sin_coinc_final = df_defunciones[df_defunciones[crear_columna].isna()][col_codigo].unique()
    if len(sin_coinc_final) > 0:
        print(f"\n   Ejemplos de códigos sin coincidencia final: {sin_coinc_final[:5]}")
    
    return df_defunciones

### 2.8. FUNCION PARA ALMACENAR RESULTADOS

In [23]:
### 2.8. FUNCION PARA GUARDAR DATAFRAME PROCESADO ###

def guardar_dataframe_procesado(df, nombre_archivo, ruta_carpeta="data/processed/"):
    """
    Guarda el DataFrame procesado en formato Parquet (óptimo para pandas).
    Si el archivo existe, lo reemplaza.
    
    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame a guardar
    nombre_archivo : str
        Nombre del archivo sin extensión (ej: 'defunciones_1979_1991')
    ruta_carpeta : str
        Ruta de la carpeta donde guardar (default: 'data/processed/')
    
    Retorna:
    --------
    str: Ruta completa del archivo guardado
    """
    from pathlib import Path
    import os
    
    # Crear carpeta si no existe
    Path(ruta_carpeta).mkdir(parents=True, exist_ok=True)
    
    # Construir ruta completa con extensión .parquet
    archivo_parquet = Path(ruta_carpeta) / f"{nombre_archivo}.parquet"
    
    # Verificar si existe
    if archivo_parquet.exists():
        print(f"   ⚠️  El archivo ya existe: {archivo_parquet}")
        print(f"   📝 Reemplazando con nueva versión...")
        # Eliminar archivo anterior
        os.remove(archivo_parquet)
    else:
        print(f"   ✨ Creando nuevo archivo: {archivo_parquet}")
    
    # Guardar en formato Parquet (más eficiente que CSV para pandas)
    df.to_parquet(archivo_parquet, index=False, compression='snappy')
    
    # Obtener tamaño del archivo
    tamaño_mb = archivo_parquet.stat().st_size / (1024 * 1024)
    
    print(f"   ✅ Archivo guardado exitosamente")
    print(f"   📊 Registros: {len(df):,}")
    print(f"   📁 Tamaño: {tamaño_mb:.2f} MB")
    print(f"   📂 Ruta: {archivo_parquet}")
    
    # Mostrar cómo leerlo después
    print(f"\n   💡 Para leer este archivo después, usa:")
    print(f"      df = pd.read_parquet('{archivo_parquet}')")
    
    return str(archivo_parquet)

## 3. EJECUTAR PROCESO PRINCIPAL

In [24]:
# =============================================================================
# 2. PROCESO PRINCIPAL
# =============================================================================

print("=" * 60)
print("ANÁLISIS EXPLORATORIO DE DATOS - DEFUNCIONES 1979-1991")
print("=" * 60)

# 2.1. Leer archivos de defunciones
print("\n1. LEYENDO ARCHIVOS DE DEFUNCIONES...")
df_defun = leer_archivos_defunciones("data/raw/Muertes", 1979, 1991)
print(f"   Total registros: {len(df_defun):,}")
print(f"   Columnas: {list(df_defun.columns)}")

# 2.2. Leer DIVIPOLA (códigos de departamentos y municipios)
print("\n2. LEYENDO DIVIPOLA...")
divipola = pd.read_csv("data/raw/Referenciales/DIVIPOLA_CentrosPoblados.csv",
                       encoding='latin-1', sep=';', dtype=str)

print(f"   Columnas disponibles: {list(divipola.columns)}")

# Identificar las columnas correctas (pueden tener nombres diferentes)
# Buscar columnas que contengan las palabras clave
col_cod_mun = [c for c in divipola.columns if 'Municipio' in c and 'digo' in c][0]
col_nom_dpto = [c for c in divipola.columns if 'Departamento' in c and 'Nombre' in c][0]
col_nom_mun = [c for c in divipola.columns if 'Municipio' in c and 'Nombre' in c][0]

print(f"   Usando columnas: {col_cod_mun}, {col_nom_dpto}, {col_nom_mun}")

# Extraer códigos de departamento y municipio del código de 5 dígitos
divipola['COD_DPTO'] = divipola[col_cod_mun].str[:2]
divipola['COD_MUNIC'] = divipola[col_cod_mun].str[2:]

divipola_merge = divipola[['COD_DPTO', 'COD_MUNIC', col_nom_dpto, col_nom_mun]].copy()
divipola_merge.columns = ['COD_DPTO', 'COD_MUNIC', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_MUNICIPIO']

# Eliminar duplicados
divipola_merge = divipola_merge.drop_duplicates(subset=['COD_DPTO', 'COD_MUNIC'])

print(f"   Registros únicos: {len(divipola_merge):,}")
print(f"   Muestra:")
print(divipola_merge.head(3)[['COD_DPTO', 'COD_MUNIC', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_MUNICIPIO']])

# 2.3. Homologar departamento y municipio de defunción
print("\n3. HOMOLOGANDO UBICACIÓN DE DEFUNCIÓN...")
print(f"   Registros iniciales: {len(df_defun):,}")

df_defun['COD_DPTO'] = df_defun['COD_DPTO'].astype(str).str.zfill(2)
df_defun['COD_MUNIC'] = df_defun['COD_MUNIC'].astype(str).str.zfill(3)
df_defun.loc[df_defun['COD_DPTO'] == '83', 'COD_DPTO'] = '18'

print(f"   Ejemplo códigos en datos: DPTO={df_defun['COD_DPTO'].iloc[0]}, MUN={df_defun['COD_MUNIC'].iloc[0]}")

# Asegurar que divipola_merge no tiene duplicados
divipola_unico = divipola_merge.drop_duplicates(subset=['COD_DPTO', 'COD_MUNIC'])
print(f"   Códigos únicos en DIVIPOLA: {len(divipola_unico):,}")

df_defun = df_defun.merge(divipola_unico, on=['COD_DPTO', 'COD_MUNIC'], how='left')
print(f"   Registros después del merge: {len(df_defun):,}")

# Completar nulos con cabeceras
cabeceras = divipola_unico[divipola_unico['COD_MUNIC'] == '001'][
    ['COD_DPTO', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_MUNICIPIO']
].drop_duplicates(subset=['COD_DPTO'])

print(f"   Cabeceras municipales: {len(cabeceras):,}")

df_defun = df_defun.merge(cabeceras, on='COD_DPTO', how='left', suffixes=('', '_CAB'))
print(f"   Registros después de cabeceras: {len(df_defun):,}")

mask = df_defun['NOMBRE_DEPARTAMENTO'].isna()
print(f"   Registros con NOMBRE_DEPARTAMENTO nulo: {mask.sum():,}")

df_defun.loc[mask, 'NOMBRE_DEPARTAMENTO'] = df_defun.loc[mask, 'NOMBRE_DEPARTAMENTO_CAB']
df_defun.loc[mask, 'NOMBRE_MUNICIPIO'] = df_defun.loc[mask, 'NOMBRE_MUNICIPIO_CAB']
df_defun.drop(['NOMBRE_DEPARTAMENTO_CAB', 'NOMBRE_MUNICIPIO_CAB'], axis=1, inplace=True)

print(f"   Coincidencias finales: {df_defun['NOMBRE_DEPARTAMENTO'].notna().sum():,}")

# 2.4. Homologar lugar de residencia
print("\n4. HOMOLOGANDO LUGAR DE RESIDENCIA...")
print(f"   Registros antes: {len(df_defun):,}")
df_defun = homologar_departamentos_residencia(df_defun, divipola_merge)
print(f"   Registros después: {len(df_defun):,}")
print(f"   Coincidencias: {(df_defun['NOMBRE_DEPARTAMENTO_RESIDENCIA'] != 'NO REGISTRA').sum():,}")

# 2.5. Crear campos descriptivos
print("\n5. CREANDO CAMPOS DESCRIPTIVOS...")
mapeos = crear_mapeo_descripciones()

df_defun['A_DEFUN_DESC'] = df_defun['A_DEFUN'].map(mapeos['A_DEFUN'])
df_defun['SEXO_DESC'] = df_defun['SEXO'].map(mapeos['SEXO'])
df_defun['EST_CIVIL_DESC'] = df_defun['EST_CIVIL'].map(mapeos['EST_CIVIL'])
df_defun['SIT_DEFUN_DESC'] = df_defun['SIT_DEFUN'].map(mapeos['SIT_DEFUN'])
df_defun['CONS_EXP_DESC'] = df_defun['CONS_EXP'].map(mapeos['CONS_EXP'])

df_defun['GRU_ED1_STR'] = df_defun['GRU_ED1'].astype(str).str.zfill(2)
df_defun['GRU_ED1_DESC'] = df_defun['GRU_ED1_STR'].map(mapeos['GRU_ED1'])

print("   Campos creados: A_DEFUN_DESC, SEXO_DESC, EST_CIVIL_DESC, SIT_DEFUN_DESC, GRU_ED1_DESC, CONS_EXP_DESC")

# 2.6. Homologar causa básica de muerte (CIE-9)
print("\n6. HOMOLOGANDO CAUSA BÁSICA (CIE-9)...")
cie = pd.read_excel("data/raw/Referenciales/CIE_9_10.xls")
cie['CIE9'] = cie['CIE9'].astype(str)

# Eliminar duplicados en CIE
cie_unico = cie.drop_duplicates(subset=['CIE9'])
cie_dict = dict(zip(cie_unico['CIE9'], cie_unico['LITERAL9']))

print("   Creando diccionario CIE-9...")
print(f"   Total códigos CIE-9 únicos: {len(cie_dict):,}")
print(f"   Registros antes del merge: {len(df_defun):,}")

# Merge directo primero
print("   Realizando cruce exacto...")
df_defun = df_defun.merge(cie_unico[['CIE9', 'LITERAL9']], 
                          left_on='C_BAS1', right_on='CIE9', how='left')
df_defun.rename(columns={'LITERAL9': 'C_BAS1_DESC'}, inplace=True)
df_defun.drop('CIE9', axis=1, inplace=True, errors='ignore')

print(f"   Registros después del merge: {len(df_defun):,}")

coincidencias_exactas = df_defun['C_BAS1_DESC'].notna().sum()
print(f"   Coincidencias exactas: {coincidencias_exactas:,} ({coincidencias_exactas/len(df_defun)*100:.2f}%)")

# Búsqueda optimizada para los restantes
sin_coinc = df_defun['C_BAS1_DESC'].isna()
if sin_coinc.sum() > 0:
    print(f"   Buscando con variaciones: {sin_coinc.sum():,} registros...")
    
    # Aplicar búsqueda vectorizada
    tqdm.pandas(desc="   Procesando")
    df_defun.loc[sin_coinc, 'C_BAS1_DESC'] = df_defun.loc[sin_coinc, 'C_BAS1'].progress_apply(
        lambda x: buscar_cie9_optimizado(x, cie_dict)
    )
    
    nuevas_coinc = df_defun.loc[sin_coinc, 'C_BAS1_DESC'].notna().sum()
    print(f"   Nuevas coincidencias: {nuevas_coinc:,}")

total_coinc = df_defun['C_BAS1_DESC'].notna().sum()
print(f"   Total coincidencias: {total_coinc:,} ({total_coinc/len(df_defun)*100:.2f}%)")

# 2.7. Homologar con Lista 105 de Colombia
print("\n7. HOMOLOGANDO CON LISTA 105 DE COLOMBIA...")
try:
    # Leer la Lista 105
    ruta_lista105 = "data/raw/Referenciales/Lista_105_Colombia_CIE9-y-CIE10.xls"
    df_lista105 = leer_lista_105(ruta_lista105)
    
    # Mostrar muestra de la Lista 105
    print(f"\n   Muestra de Lista 105 cargada:")
    print(df_lista105[['No_Lista', 'Causa']].head(5))
    
    # Homologar - crea el campo CAU_HOMOL_DESC
    df_defun = homologar_causa_lista105(df_defun, df_lista105)
    
    print(f"\n   ✅ Campo CAU_HOMOL_DESC creado exitosamente")
    
except Exception as e:
    print(f"\n   ⚠️ Error al cargar Lista 105: {e}")
    print(f"   Se continuará sin este campo")

# 2.8. Guardar DataFrame procesado
print("\n8. GENERANDO ARCHIVO PROCESADO...")
try:
    # Nombre descriptivo del archivo
    nombre_archivo = f"defunciones_{1979}_{1991}_procesado"
    
    # Guardar
    ruta_guardada = guardar_dataframe_procesado(
        df_defun, 
        nombre_archivo=nombre_archivo,
        ruta_carpeta="data/processed/"
    )
    
    print(f"\n   🎉 DataFrame guardado correctamente")
    
except Exception as e:
    print(f"\n   ❌ Error al guardar archivo: {e}")
    print(f"   El proceso continuó pero no se guardó el archivo")

ANÁLISIS EXPLORATORIO DE DATOS - DEFUNCIONES 1979-1991

1. LEYENDO ARCHIVOS DE DEFUNCIONES...


Leyendo archivos: 100%|██████████| 13/13 [00:02<00:00,  6.31it/s]


   Total registros: 1,869,025
   Columnas: ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']

2. LEYENDO DIVIPOLA...
   Columnas disponibles: ['Código_Departamento', 'Nombre_Departamento', 'Código_Municipio', 'Nombre_Municipio', 'Código_Entidad', 'Nombre_Entidad', 'Tipo', 'Longitud', 'Latitud']
   Usando columnas: Código_Municipio, Nombre_Departamento, Nombre_Municipio
   Registros únicos: 1,122
   Muestra:
   COD_DPTO COD_MUNIC NOMBRE_DEPARTAMENTO NOMBRE_MUNICIPIO
0        05       001           ANTIOQUIA         MEDELLÍN
28       05       002           ANTIOQUIA        ABEJORRAL
32       05       004           ANTIOQUIA         ABRIAQUÍ

3. HOMOLOGANDO UBICACIÓN DE DEFUNCIÓN...
   Registros iniciales: 1,869,025
   Ejemplo códigos en datos: DPTO=50, MUN=001
   Códigos únicos en DIVIPOLA: 1,122
   Registros después del merge: 1,869,025
   Cabeceras municipales: 33
   Registros despu

   Procesando: 100%|██████████| 514995/514995 [00:01<00:00, 381123.49it/s]


   Nuevas coincidencias: 449,670
   Total coincidencias: 1,803,700 (96.50%)

7. HOMOLOGANDO CON LISTA 105 DE COLOMBIA...
Leyendo archivo: data/raw/Referenciales/Lista_105_Colombia_CIE9-y-CIE10.xls
Columnas detectadas: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3']
Datos encontrados a partir de la fila: 6
Columnas identificadas:
  - Número: None
  - Causa: None
  - CIE-10: None
  - CIE-9: None
Usando posición de columnas (A, B, C, D)

✅ Archivo procesado exitosamente
   Total de causas: 105
   Rango de listas: 1 - 105

   Muestra de Lista 105 cargada:
   No_Lista                                              Causa
1         1              Enfermedades infecciosas intestinales
2         2                            Tuberculosis y secuelas
3         3  Ciertas enfermedades transmitidas por vectores...
4         4             Ciertas enfermedades inmunoprevenibles
5         5                       Septicemia, excepto neonatal

Homologando CAU_HOMOL con Lista 105 de Colombia...
   

## 4. RESULTADOS

In [25]:
# =============================================================================
# 3. RESULTADOS FINALES
# =============================================================================

print("\n" + "=" * 60)
print("RESUMEN FINAL")
print("=" * 60)
print(f"Total registros procesados: {len(df_defun):,}")
print(f"Período: 1979-1991")
print(f"Columnas totales: {len(df_defun.columns)}")
print("\nCampos principales:")
print(f"  - Ubicación defunción: {df_defun['NOMBRE_DEPARTAMENTO'].notna().sum():,}")
print(f"  - Lugar residencia: {(df_defun['NOMBRE_DEPARTAMENTO_RESIDENCIA'] != 'NO REGISTRA').sum():,}")
print(f"  - Causa básica (CIE-9): {df_defun['C_BAS1_DESC'].notna().sum():,}")
print(f"  - Campos descriptivos: 5")

print("\n✅ PROCESO COMPLETADO")
print("\nDataFrame final: df_defun")
print(f"Shape: {df_defun.shape}")

# Mostrar muestra
print("\nMuestra de datos:")
print(df_defun[['ANO', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_MUNICIPIO', 
               'SEXO_DESC', 'GRU_ED1_DESC', 'C_BAS1_DESC']].head(10))


RESUMEN FINAL
Total registros procesados: 1,869,025
Período: 1979-1991
Columnas totales: 27

Campos principales:
  - Ubicación defunción: 1,869,025
  - Lugar residencia: 1,827,548
  - Causa básica (CIE-9): 1,803,700
  - Campos descriptivos: 5

✅ PROCESO COMPLETADO

DataFrame final: df_defun
Shape: (1869025, 27)

Muestra de datos:
    ANO NOMBRE_DEPARTAMENTO   NOMBRE_MUNICIPIO  SEXO_DESC      GRU_ED1_DESC  \
0  1979                META      VILLAVICENCIO   FEMENINO   DE 35 A 39 AÑOS   
1  1979                META           CUBARRAL   FEMENINO    DE 1 A 5 MESES   
2  1979                META           CUBARRAL   FEMENINO  DE 85 Y MÁS AÑOS   
3  1979                META  SAN JUAN DE ARAMA  MASCULINO   DE 55 A 59 AÑOS   
4  1979                META      FUENTE DE ORO  MASCULINO   DE 40 A 44 AÑOS   
5  1979                META      FUENTE DE ORO   FEMENINO   DE 6 A 11 MESES   
6  1979                META      PUERTO LLERAS  MASCULINO     DE 5 A 9 AÑOS   
7  1979                META      PU

In [26]:
pd.set_option('display.max_columns', None)
df_defun.head()

,COD_DPTO,COD_MUNIC,A_DEFUN,ANO,MES,SEXO,GRU_ED1,EST_CIVIL,CODPTORE,CODMUNRE,SIT_DEFUN,C_BAS1,CONS_EXP,CAU_HOMOL,NOMBRE_DEPARTAMENTO,NOMBRE_MUNICIPIO,NOMBRE_DEPARTAMENTO_RESIDENCIA,NOMBRE_MUNICIPIO_RESIDENCIA,A_DEFUN_DESC,SEXO_DESC,EST_CIVIL_DESC,SIT_DEFUN_DESC,CONS_EXP_DESC,GRU_ED1_STR,GRU_ED1_DESC,C_BAS1_DESC,CAU_HOMOL_DESC
0,50,001,1,1979,01,2,14,1,50,001,1,6370,2,078,META,VILLAVICENCIO,META,VILLAVICENCIO,CABECERA MUNICIPAL,FEMENINO,SOLTERO,HOSPITAL O CLINICA,MEDICO NO TRATANTE,14,DE 35 A 39 AÑOS,ABORTO N.E.COMPLICADO POR INFECCION DEL TRACTO...,"Embarazo, parto y puerperio"
1,50,223,1,1979,01,2,04,1,50,223,2,2762,3,043,META,CUBARRAL,META,CUBARRAL,CABECERA MUNICIPAL,FEMENINO,SOLTERO,CASA,SIN CERTIFICACIÓN MÉDICA,04,DE 1 A 5 MESES,ACIDOSIS,Todas las demás enfermedades endocrinas y nutr...
2,50,223,1,1979,01,2,24,1,50,223,2,2762,3,043,META,CUBARRAL,META,CUBARRAL,CABECERA MUNICIPAL,FEMENINO,SOLTERO,CASA,SIN CERTIFICACIÓN MÉDICA,24,DE 85 Y MÁS AÑOS,ACIDOSIS,Todas las demás enfermedades endocrinas y nutr...
3,50,683,1,1979,01,1,18,2,50,683,2,8259,3,090,META,SAN JUAN DE ARAMA,META,SAN JUAN DE ARAMA,CABECERA MUNICIPAL,MASCULINO,CASADO,CASA,SIN CERTIFICACIÓN MÉDICA,18,DE 55 A 59 AÑOS,ACC.OTRA NATUR. O NATUR. NO ESPECIF. DE VHM/PE...,Accidentes de transporte de motor y secuelas
4,50,287,1,1979,01,1,15,1,50,287,2,9660,3,101,META,FUENTE DE ORO,META,FUENTE DE ORO,CABECERA MUNICIPAL,MASCULINO,SOLTERO,CASA,SIN CERTIFICACIÓN MÉDICA,15,DE 40 A 44 AÑOS,"ENVEN.DERIVADOS OXAZOLIDINA:PARAMETADIONA,TRIM...",Agresiones (homicidios) y secuelas
